In [ ]:
# autogluon에서 스케일링을 test에도 train으로 학습한 걸로
!pip install autogluon

In [ ]:
import sys
# import sktime
import tqdm as tq
import xgboost as xgb
import matplotlib
import seaborn as sns
import sklearn as skl
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from tqdm import tqdm
import random
import os
from sklearn.preprocessing import MaxAbsScaler

In [ ]:
def seed_everything(seed):
    random.seed(seed)
    os.environ['PYTHONHASHSEED'] = str(seed)
    np.random.seed(seed)

seed_everything(42) # Seed 고정

In [ ]:
train = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/train_pre.csv')
train_y = train['power']
train_x = train.drop(columns=['power'])
test = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/test_pre.csv')
train_y.shape

(204000,)

In [ ]:
train_x.head()

,num,temp,prec,wind,hum,day,month,week,day_hour_mean,hour_mean,hour_std,holiday,sin_time,cos_time,THI,CDH
0,1,18.6,0.0,0.9,42.0,2,6,22,1774.744615,1706.318118,446.882767,1,0.000000,1.000000,49.6576,-7.4
1,1,18.0,0.0,1.1,45.0,2,6,22,1687.347692,1622.620235,439.662704,1,0.258819,0.965926,47.7625,-15.4
2,1,17.7,0.0,1.5,45.0,2,6,22,1571.483077,1506.971294,412.071906,1,0.500000,0.866025,47.2225,-23.7
3,1,16.7,0.0,1.4,48.0,2,6,22,1522.153846,1437.365647,391.205981,1,0.707107,0.707107,44.7856,-33.0
4,1,18.4,0.0,2.8,43.0,2,6,22,1506.793846,1447.321412,381.099697,1,0.866025,0.500000,49.0061,-40.6


In [ ]:
scaler = MaxAbsScaler()
scaler.fit(train_x)
transformed_train_X = scaler.transform(train_x)
transformed_test_X = scaler.transform(test)
transformed_train_X

array([[ 0.01      ,  0.50134771,  0.        , ...,  1.        ,
         0.5496843 , -0.04394299],
       [ 0.01      ,  0.4851752 ,  0.        , ...,  0.96592583,
         0.52870651, -0.09144893],
       [ 0.01      ,  0.47708895,  0.        , ...,  0.8660254 ,
         0.52272898, -0.14073634],
       ...,
       [ 1.        ,  0.57412399,  0.        , ...,  0.70710678,
         0.71063468, -0.1324228 ],
       [ 1.        ,  0.56603774,  0.        , ...,  0.8660254 ,
         0.72034041, -0.14904988],
       [ 1.        ,  0.55795148,  0.        , ...,  0.96592583,
         0.72253327, -0.16686461]])

In [ ]:
transformed_train_X = pd.DataFrame(data = transformed_train_X, columns = train_x.columns)
transformed_train_X['power'] = train_y
transformed_train_X.head()

,num,temp,prec,wind,hum,day,month,week,day_hour_mean,hour_mean,hour_std,holiday,sin_time,cos_time,THI,CDH,power
0,0.01,0.501348,0.0,0.067669,0.42,0.333333,0.75,0.647059,0.077458,0.081894,0.113679,1.0,0.000000,1.000000,0.549684,-0.043943,1085.28
1,0.01,0.485175,0.0,0.082707,0.45,0.333333,0.75,0.647059,0.073643,0.077877,0.111842,1.0,0.258819,0.965926,0.528707,-0.091449,1047.36
2,0.01,0.477089,0.0,0.112782,0.45,0.333333,0.75,0.647059,0.068587,0.072326,0.104824,1.0,0.500000,0.866025,0.522729,-0.140736,974.88
3,0.01,0.450135,0.0,0.105263,0.48,0.333333,0.75,0.647059,0.066434,0.068986,0.099516,1.0,0.707107,0.707107,0.495754,-0.195962,953.76
4,0.01,0.495957,0.0,0.210526,0.43,0.333333,0.75,0.647059,0.065763,0.069464,0.096945,1.0,0.866025,0.500000,0.542473,-0.241093,986.40


In [ ]:
transformed_test_X = pd.DataFrame(data = transformed_test_X, columns = test.columns)
transformed_test_X.head()

,num,temp,prec,wind,hum,day,month,week,day_hour_mean,hour_mean,hour_std,holiday,sin_time,cos_time,THI,CDH
0,0.01,0.633423,0.0,0.165414,0.72,0.5,1.0,1.0,0.071044,0.081894,0.113679,0.0,0.000000,1.000000,0.645856,-0.014846
1,0.01,0.619946,0.0,0.067669,0.72,0.5,1.0,1.0,0.067652,0.077877,0.111842,0.0,0.258819,0.965926,0.635893,-0.032660
2,0.01,0.611860,0.0,0.112782,0.75,0.5,1.0,1.0,0.062461,0.072326,0.104824,0.0,0.500000,0.866025,0.640619,-0.052257
3,0.01,0.595687,0.0,0.097744,0.78,0.5,1.0,1.0,0.059889,0.068986,0.099516,0.0,0.707107,0.707107,0.641340,-0.075416
4,0.01,0.587601,0.0,0.075188,0.77,0.5,1.0,1.0,0.060304,0.069464,0.096945,0.0,0.866025,0.500000,0.630918,-0.100356


In [ ]:
transformed_train_X.to_csv('/content/drive/MyDrive/Colab Notebooks/train_final_e.csv', index = False)
transformed_test_X.to_csv('/content/drive/MyDrive/Colab Notebooks/test_final_e.csv', index = False)

In [ ]:
from autogluon.tabular import TabularDataset, TabularPredictor

In [ ]:
train_data = TabularDataset('/content/drive/MyDrive/Colab Notebooks/train_final_e.csv')
test_data = TabularDataset('/content/drive/MyDrive/Colab Notebooks/test_final_e.csv')
train_data.head()

,num,temp,prec,wind,hum,day,month,week,day_hour_mean,hour_mean,hour_std,holiday,sin_time,cos_time,THI,CDH,power
0,0.01,0.501348,0.0,0.067669,0.42,0.333333,0.75,0.647059,0.077458,0.081894,0.113679,1.0,0.000000,1.000000,0.549684,-0.043943,1085.28
1,0.01,0.485175,0.0,0.082707,0.45,0.333333,0.75,0.647059,0.073643,0.077877,0.111842,1.0,0.258819,0.965926,0.528707,-0.091449,1047.36
2,0.01,0.477089,0.0,0.112782,0.45,0.333333,0.75,0.647059,0.068587,0.072326,0.104824,1.0,0.500000,0.866025,0.522729,-0.140736,974.88
3,0.01,0.450135,0.0,0.105263,0.48,0.333333,0.75,0.647059,0.066434,0.068986,0.099516,1.0,0.707107,0.707107,0.495754,-0.195962,953.76
4,0.01,0.495957,0.0,0.210526,0.43,0.333333,0.75,0.647059,0.065763,0.069464,0.096945,1.0,0.866025,0.500000,0.542473,-0.241093,986.40


In [ ]:
label = 'power'
eval_metric = 'mape'
time_limit = 3600 * 3

predictor = TabularPredictor(
    label=label, eval_metric=eval_metric
).fit(train_data, presets='best_quality', time_limit=time_limit, num_gpus=1)

predictor.leaderboard(silent = True)

No path specified. Models will be saved in: "AutogluonModels/ag-20230728_002842/"
Presets specified: ['best_quality']
Stack configuration (auto_stack=True): num_stack_levels=1, num_bag_folds=8, num_bag_sets=20
Beginning AutoGluon training ... Time limit = 10800s
AutoGluon will save models to "AutogluonModels/ag-20230728_002842/"
AutoGluon Version:  0.8.2
Python Version:     3.10.6
Operating System:   Linux
Platform Machine:   x86_64
Platform Version:   #1 SMP Fri Jun 9 10:57:30 UTC 2023
Disk Space Avail:   146.78 GB / 179.07 GB (82.0%)
Train Data Rows:    204000
Train Data Columns: 16
Label Column: power
Preprocessing data ...
AutoGluon infers your prediction problem is: 'regression' (because dtype of label-column == float and many unique label-values observed).
	Label info (max, min, mean, stddev): (25488.4, 0.0, 2451.03646, 2440.64886)
	If 'regression' is not the correct problem_type, please manually specify the problem_type parameter during predictor init (You may specify problem_ty

,model,score_val,pred_time_val,fit_time,pred_time_val_marginal,fit_time_marginal,stack_level,can_infer,fit_order
0,XGBoost_BAG_L1,-2.864198e+12,422.206615,502.425069,422.206615,502.425069,1,True,9
1,WeightedEnsemble_L2,-2.864198e+12,422.214086,506.832129,0.007470,4.407060,2,True,12
2,LightGBM_BAG_L2,-1.449830e+13,1545.276562,7988.279492,164.588184,1045.317927,2,True,14
3,WeightedEnsemble_L3,-1.449830e+13,1545.281103,7990.383250,0.004541,2.103758,3,True,16
4,LightGBMXT_BAG_L2,-1.561588e+13,1818.127573,8733.164279,437.439194,1790.202713,2,True,13
5,RandomForestMSE_BAG_L2,-1.762199e+13,1384.547265,7599.751604,3.858886,656.790038,2,True,15
6,LightGBMLarge_BAG_L1,-1.956483e+13,205.171989,860.843144,205.171989,860.843144,1,True,11
7,LightGBMXT_BAG_L1,-2.580052e+13,409.227435,1382.750577,409.227435,1382.750577,1,True,3
8,ExtraTreesMSE_BAG_L1,-2.668795e+13,5.907079,115.610943,5.907079,115.610943,1,True,7
9,RandomForestMSE_BAG_L1,-2.680546e+13,8.320924,492.053776,8.320924,492.053776,1,True,5


In [ ]:
def convert_to_smape(mape): # mape 값을 smape로 변환
    smape = mape / 100  # 백분율을 소수점으로 변환하기 위해 100으로 나눕니다.
    output = 2 * smape / (1 + smape) * 100  # SMAPE 계산식을 적용하여 변환합니다.
    return output

In [ ]:
convert_to_smape(-14498299110248.57)

200.00000000137948

In [ ]:
model_to_use = predictor.get_model_best()
model_pred = predictor.predict(test_data, model=model_to_use)
submission = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/submission.csv')
submission['answer'] = model_pred
submission.head()

,num_date_time,answer
0,1_20220825 00,1674.184814
1,1_20220825 01,1718.019165
2,1_20220825 02,1554.509521
3,1_20220825 03,1558.203369
4,1_20220825 04,1586.328247


In [ ]:
submission.to_csv('./스케일링 + autogluon.csv', index=False)

In [ ]:
label = 'power'
eval_metric = 'mape'
time_limit = 3600 * 5

predictor = TabularPredictor(
    label=label, eval_metric=eval_metric
).fit(train_data, presets='best_quality', time_limit=time_limit, num_gpus=1)

predictor.leaderboard(silent = True)

No path specified. Models will be saved in: "AutogluonModels/ag-20230729_031022/"
Presets specified: ['best_quality']
Stack configuration (auto_stack=True): num_stack_levels=1, num_bag_folds=8, num_bag_sets=20
Beginning AutoGluon training ... Time limit = 18000s
AutoGluon will save models to "AutogluonModels/ag-20230729_031022/"
AutoGluon Version:  0.8.2
Python Version:     3.10.6
Operating System:   Linux
Platform Machine:   x86_64
Platform Version:   #1 SMP Fri Jun 9 10:57:30 UTC 2023
Disk Space Avail:   146.78 GB / 179.07 GB (82.0%)
Train Data Rows:    204000
Train Data Columns: 16
Label Column: power
Preprocessing data ...
AutoGluon infers your prediction problem is: 'regression' (because dtype of label-column == float and many unique label-values observed).
	Label info (max, min, mean, stddev): (25488.4, 0.0, 2451.03646, 2440.64886)
	If 'regression' is not the correct problem_type, please manually specify the problem_type parameter during predictor init (You may specify problem_ty

,model,score_val,pred_time_val,fit_time,pred_time_val_marginal,fit_time_marginal,stack_level,can_infer,fit_order
0,XGBoost_BAG_L2,-2.061969e+12,1710.208666,8429.485753,27.981516,93.106007,2,True,19
1,WeightedEnsemble_L3,-2.061969e+12,1710.212231,8432.528377,0.003565,3.042624,3,True,22
2,XGBoost_BAG_L1,-2.864198e+12,432.954930,517.030390,432.954930,517.030390,1,True,9
3,WeightedEnsemble_L2,-2.864198e+12,432.960336,522.233168,0.005406,5.202778,2,True,12
4,LightGBMLarge_BAG_L2,-1.577913e+13,1824.328144,9258.480855,142.100995,922.101110,2,True,21
5,LightGBMXT_BAG_L2,-1.622419e+13,2040.317459,10193.062125,358.090309,1856.682380,2,True,13
6,ExtraTreesMSE_BAG_L2,-1.662935e+13,1691.144920,8566.562554,8.917771,230.182809,2,True,17
7,LightGBM_BAG_L2,-1.747339e+13,1786.494345,9055.612114,104.267195,719.232369,2,True,14
8,RandomForestMSE_BAG_L2,-1.755956e+13,1691.899704,10400.685852,9.672554,2064.306107,2,True,15
9,CatBoost_BAG_L2,-1.881910e+13,1683.081790,8487.765890,0.854640,151.386145,2,True,16


In [ ]:
model_to_use = predictor.get_model_best()
model_pred = predictor.predict(test_data, model=model_to_use)
submission = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/submission.csv')
submission['answer'] = model_pred
submission.head()

,num_date_time,answer
0,1_20220825 00,1685.978760
1,1_20220825 01,1726.288208
2,1_20220825 02,1525.110718
3,1_20220825 03,1510.411743
4,1_20220825 04,1512.373047


In [ ]:
submission.to_csv('./스케일링 + autogluon_5.csv', index=False)

In [ ]:
pip install --upgrade autogluon

In [ ]:
from autogluon.tabular import TabularPrediction

ImportError: ignored

In [ ]:
from autogluon.tabular import TabularPrediction as task

# 모델과 학습 상태를 저장
model_path = 'ag_20230728_model/'
predictor.save(model_path)

# ...학습 중단 또는 프로그램 종료...

ImportError: ignored

In [ ]:
train_data = TabularDataset('/content/drive/MyDrive/Colab Notebooks/train_final_e.csv')
label = 'power'
eval_metric = 'mape'
time_limit = 3600 * 3

# 모델 불러오기와 이어서 학습
predictor_loaded = task.load(model_path)
predictor_loaded.fit(train_data=train_data, label=label, eval_metric=eval_metric, presets='best_quality', time_limit=time_limit, num_gpus=1)  # 이어서 1시간 동안 학습
predictor.leaderboard(silent = True)

In [ ]:
!pip install optuna

In [ ]:
train = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/train_final_e.csv')
train_y = train['power']
train.drop(['power'], axis = 1, inplace = True)

def SMAPE(true, pred):
    v = 2 * abs(pred - true) / (abs(pred) + abs(true))
    output = np.mean(v) * 100
    return output

In [ ]:
train.head()

,num,temp,prec,wind,hum,day,month,week,day_hour_mean,hour_mean,hour_std,holiday,sin_time,cos_time,THI,CDH
0,0.01,0.501348,0.0,0.067669,0.42,0.333333,0.75,0.647059,0.077458,0.081894,0.113679,1.0,0.000000,1.000000,0.549684,-0.043943
1,0.01,0.485175,0.0,0.082707,0.45,0.333333,0.75,0.647059,0.073643,0.077877,0.111842,1.0,0.258819,0.965926,0.528707,-0.091449
2,0.01,0.477089,0.0,0.112782,0.45,0.333333,0.75,0.647059,0.068587,0.072326,0.104824,1.0,0.500000,0.866025,0.522729,-0.140736
3,0.01,0.450135,0.0,0.105263,0.48,0.333333,0.75,0.647059,0.066434,0.068986,0.099516,1.0,0.707107,0.707107,0.495754,-0.195962
4,0.01,0.495957,0.0,0.210526,0.43,0.333333,0.75,0.647059,0.065763,0.069464,0.096945,1.0,0.866025,0.500000,0.542473,-0.241093


In [ ]:
import optuna
from sklearn.ensemble import ExtraTreesRegressor
from sklearn.model_selection import train_test_split

In [ ]:
# 데이터 로드 및 전처리 (X_train, X_valid, y_train, y_valid)

def objective(trial):
    n_estimators = trial.suggest_int('n_estimators', 100, 1000, 100)
    max_depth = trial.suggest_int('max_depth', 10, 30, 5)
    min_samples_split = trial.suggest_int('min_samples_split', 2, 20, 2)
    bootstrap = trial.suggest_categorical('bootstrap', [True, False])

    model = ExtraTreesRegressor(
        n_estimators=n_estimators,
        min_samples_split=min_samples_split,
        max_depth=max_depth,
        bootstrap=bootstrap
    )

    model.fit(X_train, y_train)
    preds = model.predict(X_valid)
    score = SMAPE(y_valid, preds)
    return score

X_train, X_valid, y_train, y_valid = train_test_split(train, train_y, test_size = 0.2)

study = optuna.create_study(direction='minimize')
study.optimize(objective, n_trials=300)

[I 2023-07-28 03:54:34,892] A new study created in memory with name: no-name-947900fa-e73e-4fe4-9890-743083818aa6
[I 2023-07-28 04:00:12,552] Trial 0 finished with value: 4.171416431110571 and parameters: {'n_estimators': 400, 'max_depth': 30, 'min_samples_split': 4, 'bootstrap': False}. Best is trial 0 with value: 4.171416431110571.
[I 2023-07-28 04:07:19,517] Trial 1 finished with value: 4.8087703992494655 and parameters: {'n_estimators': 1000, 'max_depth': 30, 'min_samples_split': 12, 'bootstrap': True}. Best is trial 0 with value: 4.171416431110571.
[I 2023-07-28 04:14:34,724] Trial 2 finished with value: 4.878435269109939 and parameters: {'n_estimators': 1000, 'max_depth': 20, 'min_samples_split': 8, 'bootstrap': True}. Best is trial 0 with value: 4.171416431110571.
[I 2023-07-28 04:15:39,078] Trial 3 finished with value: 4.582051213382807 and parameters: {'n_estimators': 100, 'max_depth': 30, 'min_samples_split': 14, 'bootstrap': False}. Best is trial 0 with value: 4.171416431110

KeyboardInterrupt: ignored

In [ ]:
print('Best trial:', study.best_trial.params)
print('Best score:', study.best_value)

Best trial: {'n_estimators': 400, 'max_depth': 30, 'min_samples_split': 4, 'bootstrap': False}
Best score: 4.171416431110571


In [ ]:
def objective(trial):
    n_estimators = trial.suggest_int('n_estimators', 400, 600, 10)
    max_depth = trial.suggest_int('max_depth', 25, 30, 1)
    min_samples_split = trial.suggest_int('min_samples_split', 2, 5, 1)
    bootstrap = trial.suggest_categorical('bootstrap', [True, False])

    model = ExtraTreesRegressor(
        n_estimators=n_estimators,
        min_samples_split=min_samples_split,
        max_depth=max_depth,
        bootstrap=bootstrap
    )

    model.fit(X_train, y_train)
    preds = model.predict(X_valid)
    score = SMAPE(y_valid, preds)
    return score

X_train, X_valid, y_train, y_valid = train_test_split(train, train_y, test_size = 0.2)

study = optuna.create_study(direction='minimize')
study.optimize(objective, n_trials=300)

[I 2023-07-28 06:58:17,238] A new study created in memory with name: no-name-bf0beff3-8af8-4dcc-a00f-d0ec256062ac
[I 2023-07-28 07:03:43,105] Trial 0 finished with value: 4.116255736073268 and parameters: {'n_estimators': 400, 'max_depth': 25, 'min_samples_split': 3, 'bootstrap': False}. Best is trial 0 with value: 4.116255736073268.
[I 2023-07-28 07:08:15,996] Trial 1 finished with value: 4.413358959851843 and parameters: {'n_estimators': 500, 'max_depth': 27, 'min_samples_split': 4, 'bootstrap': True}. Best is trial 0 with value: 4.116255736073268.
[I 2023-07-28 07:13:28,736] Trial 2 finished with value: 4.437417307894444 and parameters: {'n_estimators': 600, 'max_depth': 28, 'min_samples_split': 5, 'bootstrap': True}. Best is trial 0 with value: 4.116255736073268.
[I 2023-07-28 07:18:46,204] Trial 3 finished with value: 4.386833719026492 and parameters: {'n_estimators': 510, 'max_depth': 26, 'min_samples_split': 2, 'bootstrap': True}. Best is trial 0 with value: 4.116255736073268.
[

In [ ]:
print('Best trial:', study.best_trial.params)
print('Best score:', study.best_value)

In [ ]:
params ={'n_estimators': 450, 'max_depth': 30, 'min_samples_split': 3, 'bootstrap': False}
model = ExtraTreesRegressor(**params)
model.fit(train, train_y)

test = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/test_final_e.csv')

# 테스트 데이터에 대한 예측
preds = model.predict(test)

submission = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/submission.csv')
submission['answer'] = preds
submission.head()

,num_date_time,answer
0,1_20220825 00,1876.365670
1,1_20220825 01,1844.811911
2,1_20220825 02,1556.128137
3,1_20220825 03,1477.243944
4,1_20220825 04,1488.959260


In [ ]:
submission.to_csv('./스케일링 + optuna.csv', index=False)

In [ ]:
import optuna

# Study 객체 생성 및 최적화 실행
study = optuna.create_study(direction='minimize')  # 또는 'maximize'에 따라 목표에 맞게 설정
study.optimize(objective_function, n_trials=100)

# Study 객체 저장
study_name = "example_study"  # 저장할 파일 이름 지정
optuna.study.dump(study, f"{study_name}.pkl")

In [ ]:
import optuna
import numpy as np
from sklearn.linear_model import LinearRegression

In [ ]:
def objective(trial):
    # 하이퍼파라미터 탐색 대상: learning rate
    lr = trial.suggest_float('learning_rate', 1e-5, 1e-1, log=True)

    # 모델 생성
    model = LinearRegression()

    # 학습 실행
    model.fit(X, y)

    # 검증 데이터에 대한 손실 반환 (낮을수록 좋음)
    val_loss = ((model.predict(X) - y) ** 2).mean()
    return val_loss

In [ ]:
if __name__ == "__main__":
    # 학습을 저장하고 다시 불러와서 이어서 진행할 파일 이름
    study_name = "example_study"

    # 최적화 실행
    study = optuna.create_study(direction='minimize')
    study.optimize(objective, n_trials=100)

    # 최적화 결과 저장
    optuna.study.dump(study, f"{study_name}.pkl")

    # 최적화된 하이퍼파라미터 값 확인
    best_params = study.best_params
    print("Best Hyperparameters:", best_params)

    # 학습된 모델 저장
    model = LinearRegression()
    X, y = generate_data()
    model.fit(X, y)
    joblib.dump(model, "model_checkpoint.pkl")

In [ ]:
if __name__ == "__main__":
    # 저장된 Study 객체 로드
    study_name = "example_study"
    study = optuna.study.load_study(f"{study_name}.pkl")

    # 저장된 모델 로드
    model = joblib.load("model_checkpoint.pkl")

    # 이어서 최적화 실행
    study.optimize(objective, n_trials=100)

In [ ]:
# 추가할 컬럼
# 1. 체감온도 = 13.12 + 0.6215 * 기온 - 11.37 * (바람 속도^0.16) + 0.3965 * 기온 * (바람 속도^0.16)
# 2. 열지수 = -8.784695 + 1.61139411 * 기온 + 2.338549 * 상대습도 - 0.14611605 * 기온 * 상대습도 - 0.012308094 * (기온^2) - 0.016424828 * (상대습도^2) + 0.002211732 * (기온^2) * 상대습도 + 0.00072546 * 기온 * (상대습도^2) - 0.000003582 * (기온^2) * (상대습도^2)
  #  T = df['temperature_F']
  #   RH = df['humidity']
  #   df['heat_index'] = -42.379 + 2.04901523*T + 10.14333127*RH - .22475541*T*RH - .00683783*T*T - .05481717*RH*RH + .00122874*T*T*RH + .00085282*T*RH*RH - .00000199*T*T*RH*RH
  #   df['heat_index'] = (df['heat_index']-32) * 5/9
  #   df.loc[df['heat_index']<32, 'heat_index'] = 0
  #   df.loc[(df['heat_index']>=32) & (df['heat_index']<41), 'heat_index'] = 1
  #   df.loc[(df['heat_index']>=41) & (df['heat_index']<54), 'heat_index'] = 2
  #   df.loc[(df['heat_index']>=54) & (df['heat_index']<66), 'heat_index'] = 3
  #   df.loc[df['heat_index']>=66, 'heat_index'] = 4
# 3. 상대습도(%) = (현재 습도 / 최대 수증기 포화량) * 100

In [ ]:
train = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/train_pre.csv')
train_y = train['power']
train_x = train.drop(columns=['power'])
test = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/test_pre.csv')
train_y.shape

(204000,)

In [ ]:
train_x.head()

,num,temp,prec,wind,hum,day,month,week,day_hour_mean,hour_mean,hour_std,holiday,sin_time,cos_time,THI,CDH,wc_temp
0,1,18.6,0.0,0.9,42.0,2,6,22,1774.744615,1706.318118,446.882767,1,0.000000,1.000000,49.6576,-7.4,20.751584
1,1,18.0,0.0,1.1,45.0,2,6,22,1687.347692,1622.620235,439.662704,1,0.258819,0.965926,47.7625,-15.4,20.008954
2,1,17.7,0.0,1.5,45.0,2,6,22,1571.483077,1506.971294,412.071906,1,0.500000,0.866025,47.2225,-23.7,19.476910
3,1,16.7,0.0,1.4,48.0,2,6,22,1522.153846,1437.365647,391.205981,1,0.707107,0.707107,44.7856,-33.0,18.487958
4,1,18.4,0.0,2.8,43.0,2,6,22,1506.793846,1447.321412,381.099697,1,0.866025,0.500000,49.0061,-40.6,19.751534


In [ ]:
train_x['wc_temp'] = 13.12 + 0.6215 * train_x['temp'] - 11.37 * (train_x['wind'] ** 0.16) + 0.3965 * train_x['temp'] * (train_x['wind'] ** 0.16)

In [ ]:
13.12 + 0.6215 * 18.4	 - 11.37 * (2.8 ** 0.16) + 0.3965 * 18.4	 * (2.8 ** 0.16)

19.751534188979115

In [ ]:
train_x['heat_index'] = (-8.784695 + 1.61139411 * train_x['temp'] + 2.338549 * train_x['hum'] - 0.14611605 * train_x['temp'] * train_x['hum']
- 0.012308094 * (train_x['temp'] ** 2) - 0.016424828 * (train_x['hum'] ** 2) + 0.002211732 * (train_x['temp'] ** 2) * train_x['hum']
+ 0.00072546 * train_x['temp'] * (train_x['hum'] ** 2) - 0.000003582 * (train_x['temp'] ** 2) * (train_x['hum'] ** 2))

In [ ]:
train_x.head()

,num,temp,prec,wind,hum,day,month,week,day_hour_mean,hour_mean,hour_std,holiday,sin_time,cos_time,THI,CDH,wc_temp,heat_index
0,1,18.6,0.0,0.9,42.0,2,6,22,1774.744615,1706.318118,446.882767,1,0.000000,1.000000,49.6576,-7.4,20.751584,25.782737
1,1,18.0,0.0,1.1,45.0,2,6,22,1687.347692,1622.620235,439.662704,1,0.258819,0.965926,47.7625,-15.4,20.008954,26.192924
2,1,17.7,0.0,1.5,45.0,2,6,22,1571.483077,1506.971294,412.071906,1,0.500000,0.866025,47.2225,-23.7,19.476910,26.384916
3,1,16.7,0.0,1.4,48.0,2,6,22,1522.153846,1437.365647,391.205981,1,0.707107,0.707107,44.7856,-33.0,18.487958,27.193462
4,1,18.4,0.0,2.8,43.0,2,6,22,1506.793846,1447.321412,381.099697,1,0.866025,0.500000,49.0061,-40.6,19.751534,25.916568


In [ ]:
test['wc_temp'] = 13.12 + 0.6215 * test['temp'] - 11.37 * (test['wind'] ** 0.16) + 0.3965 * test['temp'] * (test['wind'] ** 0.16)
test['heat_index'] = (-8.784695 + 1.61139411 * test['temp'] + 2.338549 * test['hum'] - 0.14611605 * test['temp'] * test['hum']
- 0.012308094 * (test['temp'] ** 2) - 0.016424828 * (test['hum'] ** 2) + 0.002211732 * (test['temp'] ** 2) * test['hum']
+ 0.00072546 * test['temp'] * (test['hum'] ** 2) - 0.000003582 * (test['temp'] ** 2) * (test['hum'] ** 2))

In [ ]:
test.head()

,num,temp,prec,wind,hum,day,month,week,day_hour_mean,hour_mean,hour_std,holiday,sin_time,cos_time,THI,CDH,wc_temp,heat_index
0,1,23.5,0.0,2.2,72,3,8,34,1627.80,1706.318118,446.882767,0,0.000000,1.000000,58.3456,-2.5,25.397063,24.353333
1,1,23.0,0.0,0.9,72,3,8,34,1550.08,1622.620235,439.662704,0,0.258819,0.965926,57.4456,-5.5,25.201620,23.942876
2,1,22.7,0.0,1.5,75,3,8,34,1431.12,1506.971294,412.071906,0,0.500000,0.866025,57.8725,-8.8,24.699788,23.416634
3,1,22.1,0.0,1.3,78,3,8,34,1372.20,1437.365647,391.205981,0,0.707107,0.707107,57.9376,-12.7,24.136018,22.576014
4,1,21.8,0.0,1.0,77,3,8,34,1381.72,1447.321412,381.099697,0,0.866025,0.500000,56.9961,-16.9,23.942400,22.518742


In [ ]:
train_x.to_csv('/content/drive/MyDrive/Colab Notebooks/train_add2.csv', index = False)
test.to_csv('/content/drive/MyDrive/Colab Notebooks/test_add2.csv', index = False)

In [ ]:
scaler = MaxAbsScaler()
scaler.fit(train_x)
transformed_train_X = scaler.transform(train_x)
transformed_test_X = scaler.transform(test)
transformed_train_X

array([[ 0.01      ,  0.50134771,  0.        , ..., -0.04394299,
         0.51494701,  0.44165496],
       [ 0.01      ,  0.4851752 ,  0.        , ..., -0.09144893,
         0.49651877,  0.4486814 ],
       [ 0.01      ,  0.47708895,  0.        , ..., -0.14073634,
         0.48331621,  0.4519702 ],
       ...,
       [ 1.        ,  0.57412399,  0.        , ..., -0.1324228 ,
         0.58149583,  0.32720608],
       [ 1.        ,  0.56603774,  0.        , ..., -0.14904988,
         0.5871505 ,  0.31224595],
       [ 1.        ,  0.55795148,  0.        , ..., -0.16686461,
         0.5905227 ,  0.30142935]])

In [ ]:
transformed_train_X = pd.DataFrame(data = transformed_train_X, columns = train_x.columns)
transformed_train_X['power'] = train_y

transformed_test_X = pd.DataFrame(data = transformed_test_X, columns = test.columns)

transformed_train_X.to_csv('/content/drive/MyDrive/Colab Notebooks/train_add2_scale.csv', index = False)
transformed_test_X.to_csv('/content/drive/MyDrive/Colab Notebooks/test_add2_scale.csv', index = False)

In [ ]:
transformed_train_X.head()

,num,temp,prec,wind,hum,day,month,week,day_hour_mean,hour_mean,hour_std,holiday,sin_time,cos_time,THI,CDH,wc_temp,heat_index,power
0,0.01,0.501348,0.0,0.067669,0.42,0.333333,0.75,0.647059,0.077458,0.081894,0.113679,1.0,0.000000,1.000000,0.549684,-0.043943,0.514947,0.441655,1085.28
1,0.01,0.485175,0.0,0.082707,0.45,0.333333,0.75,0.647059,0.073643,0.077877,0.111842,1.0,0.258819,0.965926,0.528707,-0.091449,0.496519,0.448681,1047.36
2,0.01,0.477089,0.0,0.112782,0.45,0.333333,0.75,0.647059,0.068587,0.072326,0.104824,1.0,0.500000,0.866025,0.522729,-0.140736,0.483316,0.451970,974.88
3,0.01,0.450135,0.0,0.105263,0.48,0.333333,0.75,0.647059,0.066434,0.068986,0.099516,1.0,0.707107,0.707107,0.495754,-0.195962,0.458776,0.465820,953.76
4,0.01,0.495957,0.0,0.210526,0.43,0.333333,0.75,0.647059,0.065763,0.069464,0.096945,1.0,0.866025,0.500000,0.542473,-0.241093,0.490131,0.443947,986.40


In [ ]:
transformed_test_X.head()

,num,temp,prec,wind,hum,day,month,week,day_hour_mean,hour_mean,hour_std,holiday,sin_time,cos_time,THI,CDH,wc_temp,heat_index
0,0.01,0.633423,0.0,0.165414,0.72,0.5,1.0,1.0,0.071044,0.081894,0.113679,0.0,0.000000,1.000000,0.645856,-0.014846,0.630224,0.417169
1,0.01,0.619946,0.0,0.067669,0.72,0.5,1.0,1.0,0.067652,0.077877,0.111842,0.0,0.258819,0.965926,0.635893,-0.032660,0.625374,0.410138
2,0.01,0.611860,0.0,0.112782,0.75,0.5,1.0,1.0,0.062461,0.072326,0.104824,0.0,0.500000,0.866025,0.640619,-0.052257,0.612921,0.401124
3,0.01,0.595687,0.0,0.097744,0.78,0.5,1.0,1.0,0.059889,0.068986,0.099516,0.0,0.707107,0.707107,0.641340,-0.075416,0.598931,0.386724
4,0.01,0.587601,0.0,0.075188,0.77,0.5,1.0,1.0,0.060304,0.069464,0.096945,0.0,0.866025,0.500000,0.630918,-0.100356,0.594127,0.385743


In [ ]:
!unzip '/content/drive/MyDrive/Colab Notebooks/elect.zip'

Archive:  /content/drive/MyDrive/Colab Notebooks/elect.zip
  inflating: building_info.csv       
  inflating: sample_submission.csv   
  inflating: test.csv                
  inflating: train.csv               


In [ ]:
build = pd.read_csv('/content/building_info.csv')
build.head()

,건물번호,건물유형,연면적(m2),냉방면적(m2),태양광용량(kW),ESS저장용량(kWh),PCS용량(kW)
0,1,건물기타,110634.00,39570.00,-,-,-
1,2,건물기타,122233.47,99000.00,-,-,-
2,3,건물기타,171243.00,113950.00,40,-,-
3,4,건물기타,74312.98,34419.62,60,-,-
4,5,건물기타,205884.00,150000.00,-,2557,1000


In [ ]:
build.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100 entries, 0 to 99
Data columns (total 7 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   건물번호          100 non-null    int64  
 1   건물유형          100 non-null    object 
 2   연면적(m2)       100 non-null    float64
 3   냉방면적(m2)      100 non-null    float64
 4   태양광용량(kW)     100 non-null    object 
 5   ESS저장용량(kWh)  100 non-null    object 
 6   PCS용량(kW)     100 non-null    object 
dtypes: float64(2), int64(1), object(4)
memory usage: 5.6+ KB


In [ ]:
build.nunique()

건물번호            100
건물유형             12
연면적(m2)         100
냉방면적(m2)         98
태양광용량(kW)        33
ESS저장용량(kWh)      6
PCS용량(kW)         5
dtype: int64

In [ ]:
build['PCS용량(kW)'].value_counts()

-       95
500      2
1000     1
300      1
800      1
Name: PCS용량(kW), dtype: int64

In [ ]:
build['태양광용량(kW)'] = build['태양광용량(kW)'].replace('-', 0)

In [ ]:
build['ESS저장용량(kWh)'] = build['ESS저장용량(kWh)'].replace('-', 0)
build['PCS용량(kW)'] = build['PCS용량(kW)'].replace('-', 0)

In [ ]:
from sklearn.preprocessing import LabelEncoder

In [ ]:
le = LabelEncoder()
build['건물유형'] = le.fit_transform(build['건물유형'])
build['건물유형'].value_counts()

0     15
1      8
2      8
4      8
5      8
6      8
7      8
8      8
9      8
10     8
11     8
3      5
Name: 건물유형, dtype: int64

In [ ]:
build.head()

,건물번호,건물유형,연면적(m2),냉방면적(m2),태양광용량(kW),ESS저장용량(kWh),PCS용량(kW)
0,1,0,110634.00,39570.00,0,0,0
1,2,0,122233.47,99000.00,0,0,0
2,3,0,171243.00,113950.00,40,0,0
3,4,0,74312.98,34419.62,60,0,0
4,5,0,205884.00,150000.00,0,2557,1000


In [ ]:
train = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/train_add2.csv', )
test = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/test_add2.csv')

In [ ]:
train.head()

,num,temp,prec,wind,hum,day,month,week,day_hour_mean,hour_mean,hour_std,holiday,sin_time,cos_time,THI,CDH,wc_temp,heat_index
0,1,18.6,0.0,0.9,42.0,2,6,22,1774.744615,1706.318118,446.882767,1,0.000000,1.000000,49.6576,-7.4,20.751584,25.782737
1,1,18.0,0.0,1.1,45.0,2,6,22,1687.347692,1622.620235,439.662704,1,0.258819,0.965926,47.7625,-15.4,20.008954,26.192924
2,1,17.7,0.0,1.5,45.0,2,6,22,1571.483077,1506.971294,412.071906,1,0.500000,0.866025,47.2225,-23.7,19.476910,26.384916
3,1,16.7,0.0,1.4,48.0,2,6,22,1522.153846,1437.365647,391.205981,1,0.707107,0.707107,44.7856,-33.0,18.487958,27.193462
4,1,18.4,0.0,2.8,43.0,2,6,22,1506.793846,1447.321412,381.099697,1,0.866025,0.500000,49.0061,-40.6,19.751534,25.916568


In [ ]:
merged_df = pd.merge(train, build, left_on='num', right_on='건물번호', how='inner')
merged_df.head()

,num,temp,prec,wind,hum,day,month,week,day_hour_mean,hour_mean,...,CDH,wc_temp,heat_index,건물번호,건물유형,연면적(m2),냉방면적(m2),태양광용량(kW),ESS저장용량(kWh),PCS용량(kW)
0,1,18.6,0.0,0.9,42.0,2,6,22,1774.744615,1706.318118,...,-7.4,20.751584,25.782737,1,0,110634.0,39570.0,0,0,0
1,1,18.0,0.0,1.1,45.0,2,6,22,1687.347692,1622.620235,...,-15.4,20.008954,26.192924,1,0,110634.0,39570.0,0,0,0
2,1,17.7,0.0,1.5,45.0,2,6,22,1571.483077,1506.971294,...,-23.7,19.476910,26.384916,1,0,110634.0,39570.0,0,0,0
3,1,16.7,0.0,1.4,48.0,2,6,22,1522.153846,1437.365647,...,-33.0,18.487958,27.193462,1,0,110634.0,39570.0,0,0,0
4,1,18.4,0.0,2.8,43.0,2,6,22,1506.793846,1447.321412,...,-40.6,19.751534,25.916568,1,0,110634.0,39570.0,0,0,0


In [ ]:
merged_df[merged_df['num'] != merged_df['건물번호']]

,num,temp,prec,wind,hum,day,month,week,day_hour_mean,hour_mean,...,CDH,wc_temp,heat_index,건물번호,건물유형,연면적(m2),냉방면적(m2),태양광용량(kW),ESS저장용량(kWh),PCS용량(kW)


In [ ]:
merged_df.drop('건물번호', axis = 1, inplace = True)

In [ ]:
train.columns.to_list()

['num',
 'temp',
 'prec',
 'wind',
 'hum',
 'day',
 'month',
 'week',
 'day_hour_mean',
 'hour_mean',
 'hour_std',
 'holiday',
 'sin_time',
 'cos_time',
 'THI',
 'CDH',
 'wc_temp',
 'heat_index']

In [ ]:
merged_df.columns = train.columns.to_list() + ['build_type', 'space', 'cold_space', 'sun', 'ess', 'pcs']
merged_df.head()

,num,temp,prec,wind,hum,day,month,week,day_hour_mean,hour_mean,...,THI,CDH,wc_temp,heat_index,build_type,space,cold_space,sun,ess,pcs
0,1,18.6,0.0,0.9,42.0,2,6,22,1774.744615,1706.318118,...,49.6576,-7.4,20.751584,25.782737,0,110634.0,39570.0,0,0,0
1,1,18.0,0.0,1.1,45.0,2,6,22,1687.347692,1622.620235,...,47.7625,-15.4,20.008954,26.192924,0,110634.0,39570.0,0,0,0
2,1,17.7,0.0,1.5,45.0,2,6,22,1571.483077,1506.971294,...,47.2225,-23.7,19.476910,26.384916,0,110634.0,39570.0,0,0,0
3,1,16.7,0.0,1.4,48.0,2,6,22,1522.153846,1437.365647,...,44.7856,-33.0,18.487958,27.193462,0,110634.0,39570.0,0,0,0
4,1,18.4,0.0,2.8,43.0,2,6,22,1506.793846,1447.321412,...,49.0061,-40.6,19.751534,25.916568,0,110634.0,39570.0,0,0,0


In [ ]:
merged_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 204000 entries, 0 to 203999
Data columns (total 24 columns):
 #   Column         Non-Null Count   Dtype  
---  ------         --------------   -----  
 0   num            204000 non-null  int64  
 1   temp           204000 non-null  float64
 2   prec           204000 non-null  float64
 3   wind           204000 non-null  float64
 4   hum            204000 non-null  float64
 5   day            204000 non-null  int64  
 6   month          204000 non-null  int64  
 7   week           204000 non-null  int64  
 8   day_hour_mean  204000 non-null  float64
 9   hour_mean      204000 non-null  float64
 10  hour_std       204000 non-null  float64
 11  holiday        204000 non-null  int64  
 12  sin_time       204000 non-null  float64
 13  cos_time       204000 non-null  float64
 14  THI            204000 non-null  float64
 15  CDH            204000 non-null  float64
 16  wc_temp        204000 non-null  float64
 17  heat_index     204000 non-nul

In [ ]:
merged_df['sun'] = merged_df['sun'].astype(float)

In [ ]:
merged_df['sun'].value_counts()

0.00      130560
25.00       4080
20.00       4080
50.00       4080
200.00      4080
28.44       2040
3.00        2040
9.45        2040
365.00      2040
14.40       2040
65.00       2040
95.00       2040
60.45       2040
621.00      2040
100.00      2040
97.85       2040
98.28       2040
46.20       2040
35.00       2040
100.56      2040
40.00       2040
83.72       2040
30.00       2040
26.64       2040
97.65       2040
110.00      2040
70.00       2040
64.80       2040
56.00       2040
130.60      2040
447.20      2040
60.00       2040
21.60       2040
Name: sun, dtype: int64

In [ ]:
merged_df['ess'] = merged_df['ess'].astype(float)
merged_df['pcs'] = merged_df['pcs'].astype(int)

In [ ]:
merged_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 204000 entries, 0 to 203999
Data columns (total 24 columns):
 #   Column         Non-Null Count   Dtype  
---  ------         --------------   -----  
 0   num            204000 non-null  int64  
 1   temp           204000 non-null  float64
 2   prec           204000 non-null  float64
 3   wind           204000 non-null  float64
 4   hum            204000 non-null  float64
 5   day            204000 non-null  int64  
 6   month          204000 non-null  int64  
 7   week           204000 non-null  int64  
 8   day_hour_mean  204000 non-null  float64
 9   hour_mean      204000 non-null  float64
 10  hour_std       204000 non-null  float64
 11  holiday        204000 non-null  int64  
 12  sin_time       204000 non-null  float64
 13  cos_time       204000 non-null  float64
 14  THI            204000 non-null  float64
 15  CDH            204000 non-null  float64
 16  wc_temp        204000 non-null  float64
 17  heat_index     204000 non-nul

In [ ]:
build.columns = ['건물번호', 'build_type', 'space', 'cold_space', 'sun', 'ess', 'pcs']
build['sun'] = build['sun'].astype(float)
build['ess'] = build['ess'].astype(float)
build['pcs'] = build['pcs'].astype(int)
build.head()

,건물번호,build_type,space,cold_space,sun,ess,pcs
0,1,0,110634.00,39570.00,0.0,0.0,0
1,2,0,122233.47,99000.00,0.0,0.0,0
2,3,0,171243.00,113950.00,40.0,0.0,0
3,4,0,74312.98,34419.62,60.0,0.0,0
4,5,0,205884.00,150000.00,0.0,2557.0,1000


In [ ]:
build.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100 entries, 0 to 99
Data columns (total 7 columns):
 #   Column      Non-Null Count  Dtype  
---  ------      --------------  -----  
 0   건물번호        100 non-null    int64  
 1   build_type  100 non-null    int64  
 2   space       100 non-null    float64
 3   cold_space  100 non-null    float64
 4   sun         100 non-null    float64
 5   ess         100 non-null    float64
 6   pcs         100 non-null    int64  
dtypes: float64(4), int64(3)
memory usage: 5.6 KB


In [ ]:
merged_test = pd.merge(test, build, left_on='num', right_on='건물번호', how='inner')
merged_test.head()

,Unnamed: 0,num,temp,prec,wind,hum,day,month,week,day_hour_mean,...,CDH,wc_temp,heat_index,건물번호,build_type,space,cold_space,sun,ess,pcs
0,0,1,23.5,0.0,2.2,72,3,8,34,1627.80,...,-2.5,25.397063,24.353333,1,0,110634.0,39570.0,0.0,0.0,0
1,1,1,23.0,0.0,0.9,72,3,8,34,1550.08,...,-5.5,25.201620,23.942876,1,0,110634.0,39570.0,0.0,0.0,0
2,2,1,22.7,0.0,1.5,75,3,8,34,1431.12,...,-8.8,24.699788,23.416634,1,0,110634.0,39570.0,0.0,0.0,0
3,3,1,22.1,0.0,1.3,78,3,8,34,1372.20,...,-12.7,24.136018,22.576014,1,0,110634.0,39570.0,0.0,0.0,0
4,4,1,21.8,0.0,1.0,77,3,8,34,1381.72,...,-16.9,23.942400,22.518742,1,0,110634.0,39570.0,0.0,0.0,0


In [ ]:
merged_test.drop('Unnamed: 0', axis = 1, inplace = True)

In [ ]:
merged_test.head()

,num,temp,prec,wind,hum,day,month,week,day_hour_mean,hour_mean,...,THI,CDH,wc_temp,heat_index,build_type,space,cold_space,sun,ess,pcs
0,1,23.5,0.0,2.2,72,3,8,34,1627.80,1706.318118,...,58.3456,-2.5,25.397063,24.353333,0,110634.0,39570.0,0.0,0.0,0
1,1,23.0,0.0,0.9,72,3,8,34,1550.08,1622.620235,...,57.4456,-5.5,25.201620,23.942876,0,110634.0,39570.0,0.0,0.0,0
2,1,22.7,0.0,1.5,75,3,8,34,1431.12,1506.971294,...,57.8725,-8.8,24.699788,23.416634,0,110634.0,39570.0,0.0,0.0,0
3,1,22.1,0.0,1.3,78,3,8,34,1372.20,1437.365647,...,57.9376,-12.7,24.136018,22.576014,0,110634.0,39570.0,0.0,0.0,0
4,1,21.8,0.0,1.0,77,3,8,34,1381.72,1447.321412,...,56.9961,-16.9,23.942400,22.518742,0,110634.0,39570.0,0.0,0.0,0


In [ ]:
merged_test.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 16800 entries, 0 to 16799
Data columns (total 24 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   num            16800 non-null  int64  
 1   temp           16800 non-null  float64
 2   prec           16800 non-null  float64
 3   wind           16800 non-null  float64
 4   hum            16800 non-null  int64  
 5   day            16800 non-null  int64  
 6   month          16800 non-null  int64  
 7   week           16800 non-null  int64  
 8   day_hour_mean  16800 non-null  float64
 9   hour_mean      16800 non-null  float64
 10  hour_std       16800 non-null  float64
 11  holiday        16800 non-null  int64  
 12  sin_time       16800 non-null  float64
 13  cos_time       16800 non-null  float64
 14  THI            16800 non-null  float64
 15  CDH            16800 non-null  float64
 16  wc_temp        16800 non-null  float64
 17  heat_index     16800 non-null  float64
 18  build_

In [ ]:
merged_df.to_csv('/content/drive/MyDrive/Colab Notebooks/train_merge.csv', index = False)
merged_test.to_csv('/content/drive/MyDrive/Colab Notebooks/test_merge.csv', index = False)

In [ ]:
test = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/test_add2.csv')
test.head()

,Unnamed: 0,num,temp,prec,wind,hum,day,month,week,day_hour_mean,hour_mean,hour_std,holiday,sin_time,cos_time,THI,CDH,wc_temp,heat_index
0,0,1,23.5,0.0,2.2,72,3,8,34,1627.80,1706.318118,446.882767,0,0.000000,1.000000,58.3456,-2.5,25.397063,24.353333
1,1,1,23.0,0.0,0.9,72,3,8,34,1550.08,1622.620235,439.662704,0,0.258819,0.965926,57.4456,-5.5,25.201620,23.942876
2,2,1,22.7,0.0,1.5,75,3,8,34,1431.12,1506.971294,412.071906,0,0.500000,0.866025,57.8725,-8.8,24.699788,23.416634
3,3,1,22.1,0.0,1.3,78,3,8,34,1372.20,1437.365647,391.205981,0,0.707107,0.707107,57.9376,-12.7,24.136018,22.576014
4,4,1,21.8,0.0,1.0,77,3,8,34,1381.72,1447.321412,381.099697,0,0.866025,0.500000,56.9961,-16.9,23.942400,22.518742


In [ ]:
test.drop('Unnamed: 0', inplace = True, axis = 1)

In [ ]:
test.head()

,num,temp,prec,wind,hum,day,month,week,day_hour_mean,hour_mean,hour_std,holiday,sin_time,cos_time,THI,CDH,wc_temp,heat_index
0,1,23.5,0.0,2.2,72,3,8,34,1627.80,1706.318118,446.882767,0,0.000000,1.000000,58.3456,-2.5,25.397063,24.353333
1,1,23.0,0.0,0.9,72,3,8,34,1550.08,1622.620235,439.662704,0,0.258819,0.965926,57.4456,-5.5,25.201620,23.942876
2,1,22.7,0.0,1.5,75,3,8,34,1431.12,1506.971294,412.071906,0,0.500000,0.866025,57.8725,-8.8,24.699788,23.416634
3,1,22.1,0.0,1.3,78,3,8,34,1372.20,1437.365647,391.205981,0,0.707107,0.707107,57.9376,-12.7,24.136018,22.576014
4,1,21.8,0.0,1.0,77,3,8,34,1381.72,1447.321412,381.099697,0,0.866025,0.500000,56.9961,-16.9,23.942400,22.518742


In [ ]:
test.to_csv('/content/drive/MyDrive/Colab Notebooks/test_add2.csv', index = False)

In [ ]:
test = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/test_add2.csv')
test.head()

,num,temp,prec,wind,hum,day,month,week,day_hour_mean,hour_mean,hour_std,holiday,sin_time,cos_time,THI,CDH,wc_temp,heat_index
0,1,23.5,0.0,2.2,72,3,8,34,1627.80,1706.318118,446.882767,0,0.000000,1.000000,58.3456,-2.5,25.397063,24.353333
1,1,23.0,0.0,0.9,72,3,8,34,1550.08,1622.620235,439.662704,0,0.258819,0.965926,57.4456,-5.5,25.201620,23.942876
2,1,22.7,0.0,1.5,75,3,8,34,1431.12,1506.971294,412.071906,0,0.500000,0.866025,57.8725,-8.8,24.699788,23.416634
3,1,22.1,0.0,1.3,78,3,8,34,1372.20,1437.365647,391.205981,0,0.707107,0.707107,57.9376,-12.7,24.136018,22.576014
4,1,21.8,0.0,1.0,77,3,8,34,1381.72,1447.321412,381.099697,0,0.866025,0.500000,56.9961,-16.9,23.942400,22.518742


In [ ]:
train_y.head()

0    1085.28
1    1047.36
2     974.88
3     953.76
4     986.40
Name: power, dtype: float64

In [ ]:
train = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/train_merge.csv')
test = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/test_merge.csv')
train['power'] = train_y
train.head()

,num,temp,prec,wind,hum,day,month,week,day_hour_mean,hour_mean,...,CDH,wc_temp,heat_index,build_type,space,cold_space,sun,ess,pcs,power
0,1,18.6,0.0,0.9,42.0,2,6,22,1774.744615,1706.318118,...,-7.4,20.751584,25.782737,0,110634.0,39570.0,0.0,0.0,0,1085.28
1,1,18.0,0.0,1.1,45.0,2,6,22,1687.347692,1622.620235,...,-15.4,20.008954,26.192924,0,110634.0,39570.0,0.0,0.0,0,1047.36
2,1,17.7,0.0,1.5,45.0,2,6,22,1571.483077,1506.971294,...,-23.7,19.476910,26.384916,0,110634.0,39570.0,0.0,0.0,0,974.88
3,1,16.7,0.0,1.4,48.0,2,6,22,1522.153846,1437.365647,...,-33.0,18.487958,27.193462,0,110634.0,39570.0,0.0,0.0,0,953.76
4,1,18.4,0.0,2.8,43.0,2,6,22,1506.793846,1447.321412,...,-40.6,19.751534,25.916568,0,110634.0,39570.0,0.0,0.0,0,986.40


In [ ]:
train.to_csv('/content/drive/MyDrive/Colab Notebooks/train_merge.csv', index = False)

In [ ]:
train = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/train_merge.csv')

In [ ]:
train.drop('Unnamed: 0', axis = 1, inplace = True)
train.head()

,num,temp,prec,wind,hum,day,month,week,day_hour_mean,hour_mean,...,CDH,wc_temp,heat_index,build_type,space,cold_space,sun,ess,pcs,power
0,1,18.6,0.0,0.9,42.0,2,6,22,1774.744615,1706.318118,...,-7.4,20.751584,25.782737,0,110634.0,39570.0,0.0,0.0,0,1085.28
1,1,18.0,0.0,1.1,45.0,2,6,22,1687.347692,1622.620235,...,-15.4,20.008954,26.192924,0,110634.0,39570.0,0.0,0.0,0,1047.36
2,1,17.7,0.0,1.5,45.0,2,6,22,1571.483077,1506.971294,...,-23.7,19.476910,26.384916,0,110634.0,39570.0,0.0,0.0,0,974.88
3,1,16.7,0.0,1.4,48.0,2,6,22,1522.153846,1437.365647,...,-33.0,18.487958,27.193462,0,110634.0,39570.0,0.0,0.0,0,953.76
4,1,18.4,0.0,2.8,43.0,2,6,22,1506.793846,1447.321412,...,-40.6,19.751534,25.916568,0,110634.0,39570.0,0.0,0.0,0,986.40


In [ ]:
train.to_csv('/content/drive/MyDrive/Colab Notebooks/train_merge.csv', index = False)
train = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/train_merge.csv')

In [ ]:
train.head()

,num,temp,prec,wind,hum,day,month,week,day_hour_mean,hour_mean,...,CDH,wc_temp,heat_index,build_type,space,cold_space,sun,ess,pcs,power
0,1,18.6,0.0,0.9,42.0,2,6,22,1774.744615,1706.318118,...,-7.4,20.751584,25.782737,0,110634.0,39570.0,0.0,0.0,0,1085.28
1,1,18.0,0.0,1.1,45.0,2,6,22,1687.347692,1622.620235,...,-15.4,20.008954,26.192924,0,110634.0,39570.0,0.0,0.0,0,1047.36
2,1,17.7,0.0,1.5,45.0,2,6,22,1571.483077,1506.971294,...,-23.7,19.476910,26.384916,0,110634.0,39570.0,0.0,0.0,0,974.88
3,1,16.7,0.0,1.4,48.0,2,6,22,1522.153846,1437.365647,...,-33.0,18.487958,27.193462,0,110634.0,39570.0,0.0,0.0,0,953.76
4,1,18.4,0.0,2.8,43.0,2,6,22,1506.793846,1447.321412,...,-40.6,19.751534,25.916568,0,110634.0,39570.0,0.0,0.0,0,986.40


In [ ]:
train = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/train_add2.csv')

In [ ]:
train['power'] = train_y
train.head()

,num,temp,prec,wind,hum,day,month,week,day_hour_mean,hour_mean,hour_std,holiday,sin_time,cos_time,THI,CDH,wc_temp,heat_index,power
0,1,18.6,0.0,0.9,42.0,2,6,22,1774.744615,1706.318118,446.882767,1,0.000000,1.000000,49.6576,-7.4,20.751584,25.782737,1085.28
1,1,18.0,0.0,1.1,45.0,2,6,22,1687.347692,1622.620235,439.662704,1,0.258819,0.965926,47.7625,-15.4,20.008954,26.192924,1047.36
2,1,17.7,0.0,1.5,45.0,2,6,22,1571.483077,1506.971294,412.071906,1,0.500000,0.866025,47.2225,-23.7,19.476910,26.384916,974.88
3,1,16.7,0.0,1.4,48.0,2,6,22,1522.153846,1437.365647,391.205981,1,0.707107,0.707107,44.7856,-33.0,18.487958,27.193462,953.76
4,1,18.4,0.0,2.8,43.0,2,6,22,1506.793846,1447.321412,381.099697,1,0.866025,0.500000,49.0061,-40.6,19.751534,25.916568,986.40


In [ ]:
train.to_csv('/content/drive/MyDrive/Colab Notebooks/train_add2.csv', index = False)
train = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/train_add2.csv')
train.head()

,num,temp,prec,wind,hum,day,month,week,day_hour_mean,hour_mean,hour_std,holiday,sin_time,cos_time,THI,CDH,wc_temp,heat_index,power
0,1,18.6,0.0,0.9,42.0,2,6,22,1774.744615,1706.318118,446.882767,1,0.000000,1.000000,49.6576,-7.4,20.751584,25.782737,1085.28
1,1,18.0,0.0,1.1,45.0,2,6,22,1687.347692,1622.620235,439.662704,1,0.258819,0.965926,47.7625,-15.4,20.008954,26.192924,1047.36
2,1,17.7,0.0,1.5,45.0,2,6,22,1571.483077,1506.971294,412.071906,1,0.500000,0.866025,47.2225,-23.7,19.476910,26.384916,974.88
3,1,16.7,0.0,1.4,48.0,2,6,22,1522.153846,1437.365647,391.205981,1,0.707107,0.707107,44.7856,-33.0,18.487958,27.193462,953.76
4,1,18.4,0.0,2.8,43.0,2,6,22,1506.793846,1447.321412,381.099697,1,0.866025,0.500000,49.0061,-40.6,19.751534,25.916568,986.40


In [ ]:
train = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/train_merge.csv')
train_x = train.drop('power', axis = 1)
train_x.head()

,num,temp,prec,wind,hum,day,month,week,day_hour_mean,hour_mean,...,THI,CDH,wc_temp,heat_index,build_type,space,cold_space,sun,ess,pcs
0,1,18.6,0.0,0.9,42.0,2,6,22,1774.744615,1706.318118,...,49.6576,-7.4,20.751584,25.782737,0,110634.0,39570.0,0.0,0.0,0
1,1,18.0,0.0,1.1,45.0,2,6,22,1687.347692,1622.620235,...,47.7625,-15.4,20.008954,26.192924,0,110634.0,39570.0,0.0,0.0,0
2,1,17.7,0.0,1.5,45.0,2,6,22,1571.483077,1506.971294,...,47.2225,-23.7,19.476910,26.384916,0,110634.0,39570.0,0.0,0.0,0
3,1,16.7,0.0,1.4,48.0,2,6,22,1522.153846,1437.365647,...,44.7856,-33.0,18.487958,27.193462,0,110634.0,39570.0,0.0,0.0,0
4,1,18.4,0.0,2.8,43.0,2,6,22,1506.793846,1447.321412,...,49.0061,-40.6,19.751534,25.916568,0,110634.0,39570.0,0.0,0.0,0


In [ ]:
test.head()

,num,temp,prec,wind,hum,day,month,week,day_hour_mean,hour_mean,...,THI,CDH,wc_temp,heat_index,build_type,space,cold_space,sun,ess,pcs
0,1,23.5,0.0,2.2,72,3,8,34,1627.80,1706.318118,...,58.3456,-2.5,25.397063,24.353333,0,110634.0,39570.0,0.0,0.0,0
1,1,23.0,0.0,0.9,72,3,8,34,1550.08,1622.620235,...,57.4456,-5.5,25.201620,23.942876,0,110634.0,39570.0,0.0,0.0,0
2,1,22.7,0.0,1.5,75,3,8,34,1431.12,1506.971294,...,57.8725,-8.8,24.699788,23.416634,0,110634.0,39570.0,0.0,0.0,0
3,1,22.1,0.0,1.3,78,3,8,34,1372.20,1437.365647,...,57.9376,-12.7,24.136018,22.576014,0,110634.0,39570.0,0.0,0.0,0
4,1,21.8,0.0,1.0,77,3,8,34,1381.72,1447.321412,...,56.9961,-16.9,23.942400,22.518742,0,110634.0,39570.0,0.0,0.0,0


In [ ]:
train_x.columns

Index(['num', 'temp', 'prec', 'wind', 'hum', 'day', 'month', 'week',
       'day_hour_mean', 'hour_mean', 'hour_std', 'holiday', 'sin_time',
       'cos_time', 'THI', 'CDH', 'wc_temp', 'heat_index', 'build_type',
       'space', 'cold_space', 'sun', 'ess', 'pcs'],
      dtype='object')

In [ ]:
test.columns

Index(['num', 'temp', 'prec', 'wind', 'hum', 'day', 'month', 'week',
       'day_hour_mean', 'hour_mean', 'hour_std', 'holiday', 'sin_time',
       'cos_time', 'THI', 'CDH', 'wc_temp', 'heat_index', 'build_type',
       'space', 'cold_space', 'sun', 'ess', 'pcs'],
      dtype='object')

In [ ]:
scaler = MaxAbsScaler()
scaler.fit(train_x)
transformed_train_X = scaler.transform(train_x)
transformed_test_X = scaler.transform(test)
transformed_train_X

array([[0.01      , 0.50134771, 0.        , ..., 0.        , 0.        ,
        0.        ],
       [0.01      , 0.4851752 , 0.        , ..., 0.        , 0.        ,
        0.        ],
       [0.01      , 0.47708895, 0.        , ..., 0.        , 0.        ,
        0.        ],
       ...,
       [1.        , 0.57412399, 0.        , ..., 0.        , 0.        ,
        0.        ],
       [1.        , 0.56603774, 0.        , ..., 0.        , 0.        ,
        0.        ],
       [1.        , 0.55795148, 0.        , ..., 0.        , 0.        ,
        0.        ]])

In [ ]:
transformed_train_X = pd.DataFrame(data = transformed_train_X, columns = train_x.columns)
transformed_train_X['power'] = train_y

transformed_test_X = pd.DataFrame(data = transformed_test_X, columns = test.columns)

transformed_train_X.to_csv('/content/drive/MyDrive/Colab Notebooks/train_merge_scale.csv', index = False)
transformed_test_X.to_csv('/content/drive/MyDrive/Colab Notebooks/test_merge_scale.csv', index = False)